In [103]:
# Passo a Passo:
# 1. Abrir a planilha e preparar as informações
# 2. Abrir o Navegador e o site
# 3. Escolher o estado
# 4. Preencher 'Nome da parte' de acordo a informação da planilha 
# 5. Preencher 'Advogado' de acordo a informação da planilha 
# 6. Preencher 'Número do processo' de acordo a informação da planilha
# 7. Clicar em Pesquisar
# 8. Selecionar OK no POP-UP
# 9. Aguardar o resultado do POP-UP
# 10. Pegar informações do resultado da pesquisa no POP-UP
# 11. Colocar numa nova planilha o resultado da pesquisa
# 12. Repetir até acabar


In [104]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pandas as pd
from time import sleep


In [ ]:
# 1. Abrir o Navegador e o site
navegador = webdriver.Chrome()
navegador.get(r'C:\Users\Alan\Downloads\Arquivos Projeto Consulta Juridica\Arquivos\index.html')
botao = navegador.find_element(By.XPATH, '/html/body/div/div/button')
# Mover o mouse até a opção de abrir a lista de botões
ActionChains(navegador).move_to_element(botao).perform()
sleep(1)

In [ ]:
# 2. Abrir a planilha e preparar as informações
planilha_processos = pd.read_excel(r'C:\Users\Alan\Downloads\Arquivos Projeto Consulta Juridica\Arquivos\Processos.xlsx')

# Percorrer as linhas da planilha 
for linha in planilha_processos.index: # Percorrendo pelo indice
    cidade = planilha_processos.loc[linha, 'Cidade'] # Pegando valor das cidades e colocando numa variável
    
    # Clicando no campo cidade no site de acordo com cada linha da planilha
    botao_df = navegador.find_element(By.PARTIAL_LINK_TEXT, cidade)
    botao_df.click()
    
    # Lista das Abas
    aba_pesquisa = navegador.window_handles[0]
    aba_nova = navegador.window_handles[1]

    # Mudando para nova aba
    navegador.switch_to.window(aba_nova) 
    
    # Preenchendo em cada campo as informações necessarias
    navegador.find_element(By.ID, 'nome').send_keys(planilha_processos.loc[linha, 'Nome'])
    navegador.find_element(By.ID, 'advogado').send_keys(planilha_processos.loc[linha, 'Advogado'])
    navegador.find_element(By.ID, 'numero').send_keys(planilha_processos.loc[linha, 'Processo'])

    navegador.find_element(By.CLASS_NAME, 'registerbtn').click() # Clicando em Pesquisar
    
    # Primeiro Alerta "Buscar no nosso Banco de Dados"
    alerta = navegador.switch_to.alert
    sleep(1)
    alerta.accept()
    
    # Esperar pro segundo Alerta
    # Criando um loop enquanto espera o 2° Alerta
    while True:
        try:
            alerta2 = navegador.switch_to.alert # Quando encontrar o alerta para o loop
            break
        except:
            sleep(1)
    
    texto_alerta2 = alerta2.text # Pegando texto do 2 alerta
    
    # Criando uma condição, se tiver processo preenhcer a planilha com 'sim', senão, preencher com 'não'
    
    # Preencher linha por linha o resultado da pesquisa:
    if 'Processo encontrado com sucesso' in texto_alerta2:
        planilha_processos.loc[linha, 'Status'] = 'Processo Encontrado' # Lembrete: Quando preencher com resultado, preencher linha por linha com o .loc[linha, coluna]
        alerta2.accept()
        print('Processo Encontrado')
    else:
        planilha_processos.loc[linha, 'Status'] = 'Processo não Encontrado' # Lembrete: Quando preencher com resultado, preencher linha por linha com o .loc[linha, coluna]
        alerta2.accept()
        print('Processo não Encontrado')

    # Fechando a Aba Atual 
    navegador.close()
    # Mudando para a Aba de Pesquisa
    navegador.switch_to.window(aba_pesquisa)

    # Volta pro Incio do for e refaz todo passo a passo
    
navegador.quit() # Fechando o navegador

In [ ]:
# Salvando planilha em uma cópia com outro nome
planilha_processos.to_excel('Processos Atualizados.xlsx')